In [1]:
%pip install optuna cebra==0.4.0 matplotlib==3.9.2 numpy pandas scipy seaborn umap_learn pyspark python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append("/main/external/dimensionality-reduction")

In [3]:
import cebra
import torch
import torch.utils
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from utils import TensorDataset, SimpleTensorDataset, SupervisedNNSolver, RDDDataset
import dotenv
import os
dotenv.load_dotenv()
dotenv.load_dotenv("/main/external/dimensionality-reduction/.env")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


MAX_MEMORY = "120g"

spark = SparkSession \
    .builder \
    .appName("UranusCluster") \
    .appName("Foo") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.memory.offHeap.enabled",True)\
    .config("spark.memory.offHeap.size","16g")   \
    .getOrCreate()

# Verify the SparkContext
print(spark.sparkContext.getConf().getAll())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 12:05:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.driver.memory', '120g'), ('spark.app.id', 'local-1734350752206'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'), ('spark.driver.host', 'b918cc30

# Data Load

In [5]:
df = spark.read.format("parquet").load("/main/external/data/transformed")\
    .select(["index","neural_data", "positional_encoding", "file_name"])
# df_iterator = df.rdd.mapPartitions(lambda part: [list(part)]).toLocalIterator()
df.show(5)

+------------+--------------------+--------------------+--------------------+
|       index|         neural_data| positional_encoding|           file_name|
+------------+--------------------+--------------------+--------------------+
|317827579904|[0.00353814595291...|0.014736815295795708|ID18150/Day6/Data...|
|317827707084|[7.32697662897408...|0.002792727523517...|ID18150/Day6/Data...|
|317827579905|[0.00376488942492...|0.014736815295795708|ID18150/Day6/Data...|
|317827707085|[5.27972821146249...|0.002792727523517...|ID18150/Day6/Data...|
|317827579906|[0.00368517413153...| 0.01473313965622418|ID18150/Day6/Data...|
+------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [6]:
n_neurons = max([row.neural_size for row in df.select(F.max(F.size("neural_data")).alias("neural_size")).collect()])
n_rows = df.count()

In [7]:
indices = df.select('index').rdd.flatMap(lambda x: x).collect()
len(indices), max(indices), min(indices)

(5308650, 395137123064, 0)

In [ ]:
kf = KFold(n_splits = 3)
for train_indices, test_indices in kf.split(indices):
    train_indices = [[indices[i]] for i in train_indices.tolist()]
    test_indices  = [[indices[i]] for i in test_indices.tolist()]
    df_train_index = spark.createDataFrame(train_indices[:], ["index"])
    df_test_index  = spark.createDataFrame(test_indices[:], ["index"])
    df_train = df_train_index.join(df, how = "left", on = "index")
    df_test  = df_test_index.join(df, how = "left", on = "index")
    df_train.show()
    df_test.show()

PySparkTypeError: [CANNOT_INFER_SCHEMA_FOR_TYPE] Can not infer schema for type: `int`.

In [13]:
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(numFolds=3)

In [14]:
cv.fit(df)

KeyError: Param(parent='CrossValidator_9540bbf0a8bc', name='estimator', doc='estimator to be cross-validated')

In [ ]:
delta = 5000
def get_x_ticks(L:int):
    x_ticks = np.arange(0,L,delta)
    x_tick_labels = [f"{t[i]/100:.2f}" for i in x_ticks]
    return x_ticks, x_tick_labels

In [ ]:
"""
- Choose number of folds
- Shuffle data?
- Get indices for test and train
- Iterate over partitions
"""

# CEBRA Encoder

# Models

In [ ]:
class ModelPipeline:
    def __init__(self,
        model_name,
        train_iterator,
        test_iterator,
        num_units,
        latent_dimension = 8):
        self.model_name = model_name
        self.train_iterator = train_iterator
        self.test_iterator = test_iterator
        self.latent_dimension = latent_dimension
        self.num_units = num_units
        with device:
            self.model = cebra.models.init(
                name = model_name,
                num_neurons = self.train_dataset.neural.shape[1],
                num_units = num_units,
                num_output = latent_dimension
            )
            self.train_dataset.configure_for(self.model)
            self.test_dataset.configure_for(self.model)
    def train_embedding(self, learning_rate, batch_size, steps = 1000, verbose = True):
        self.batch_size = batch_size
        self.steps = steps
        self.learning_rate = learning_rate
        self.steps = steps
        with device:
            criterion = cebra.models.criterions.LearnableCosineInfoNCE()
            optimizer = torch.optim.Adam(
                list(self.model.parameters()) + list(criterion.parameters()),
                lr = learning_rate
            )
            self.embedding_solver = cebra.solver.SingleSessionSolver(
                model = self.model,
                criterion = criterion,
                optimizer = optimizer,
                tqdm_on = verbose
            )
            train_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.train_dataset,
                num_steps = steps,
                batch_size = batch_size,
                prior = "empirical"
            )
            self.embedding_solver.fit(loader=train_loader)
    def train_decoder(self, verbose = True):
        with device:
            self.simple_train_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_train_tensor.type(torch.FloatTensor),
                    labels = self.y_train_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                ),
                batch_size = self.batch_size,
                shuffle = True
            )
            self.binaryClassifier = torch.nn.Sequential(
                self.model,
                torch.nn.Linear(self.latent_dimension,self.latent_dimension),
                torch.nn.GELU(),
                torch.nn.Linear(self.latent_dimension,1),
                torch.nn.GELU(),
                torch.nn.Linear(1,1) #Logit output
            ).to(device)
            self.decoder_solver = SupervisedNNSolver(
                model = self.binaryClassifier,
                criterion = torch.nn.BCEWithLogitsLoss(),
                optimizer = torch.optim.Adam(self.binaryClassifier.parameters(), lr = self.learning_rate)
            )
            self.decoder_solver.fit(self.simple_train_dataloader, num_steps = self.steps)
    def train(self, learning_rate, batch_size, steps, verbose = True):
        self.train_embedding(learning_rate, batch_size, steps, verbose)
        self.train_decoder(verbose = verbose)
    def score(self, verbose = True):
        with device:
            test_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.test_dataset,
                num_steps = self.steps,
                batch_size = self.batch_size,
                prior = "empirical"
            )
            self.embedding_score = self.embedding_solver.validation(test_loader)
            self.simple_test_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_test_tensor.type(torch.FloatTensor),
                    labels = self.y_test_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                    ),
                batch_size = self.batch_size,
                shuffle = True
                )
            self.decoder_score = self.decoder_solver.validation(self.simple_test_dataloader)['total']
            try:
                # Fit an exponential decay to the history and get the R2 value
                def exp_decay(x, a, b, c):
                    return a * np.exp(-b * x) + c

                # Fit the exponential decay to the solver history
                x_data = np.arange(len(self.embedding_solver.history))
                y_data = self.embedding_solver.history
                y0 = y_data[0]
                yf = y_data[-1]
                popt, _ = curve_fit(exp_decay, x_data, y_data, p0=(y0 - yf, 1e-6, yf))

                # Calculate the R2 value
                y_pred = exp_decay(x_data, *popt)
                r2 = r2_score(y_data, y_pred)
                if verbose:
                    print(f"Expontial decay R2: {r2}, embedding score: {self.embedding_score}, decoder score: {self.decoder_score}")
                return -r2 + self.embedding_score + 2*self.decoder_score
            except:
                return float("inf")
        

In [ ]:
pipeline = ModelPipeline(
    "offset36-model",
    X_train_tensor=X_train_tensor,
    y_train_tensor=y_train_tensor,
    X_test_tensor=X_test_tensor,
    y_test_tensor=y_test_tensor,
    latent_dimension=8,
    num_units=2
)
pipeline.train(learning_rate = 1e-3, batch_size = 12, steps = 10)
pipeline.score()

pos: -0.9850 neg:  3.4666 total:  2.4816 temperature:  0.9968: 100%|██████████| 10/10 [00:01<00:00,  5.97it/s]
100%|██████████| 10/10 [00:00<00:00, 21.33it/s]


Expontial decay R2: 0.1465373380689179, embedding score: 2.484728240966797, decoder score: 0.8064513206481934


3.951093544194266

In [ ]:
models_to_try = ('offset10-model', 'offset36-model', 'offset1-model')
models_to_try

('offset10-model', 'offset36-model', 'offset1-model', 'offset5-model')

In [ ]:
def experiment(trial):
    model_name = trial.suggest_categorical("model_name", models_to_try)
    num_units = trial.suggest_int("num_units", 1, X_train.shape[1])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int("batch_size", 50, 256, log=True)
    model = ModelPipeline(
        model_name,
        X_train_tensor=X_train_tensor,
        y_train_tensor=y_train_tensor,
        X_test_tensor=X_test_tensor,
        y_test_tensor=y_test_tensor,
        num_units=num_units,
        latent_dimension=8
    )
    model.train(learning_rate, batch_size, 500)
    score = model.score()
    
    # Save the best model to disk
    if not hasattr(experiment, "best_score") or score < experiment.best_score:
        experiment.best_score = score
        experiment.pipeline = model
        torch.save(model.model.state_dict(), "./data/models/best_model.pth")
    
    return score

In [ ]:
study = optuna.create_study(storage="sqlite:///data/optuna.db", study_name="cebra_offsets", direction="minimize", load_if_exists=True)
study.optimize(experiment, n_trials=100)

study.best_params

[I 2024-11-29 15:16:13,326] Using an existing study with name 'cebra_offsets' instead of creating a new one.
pos: -0.9735 neg:  5.9531 total:  4.9797 temperature:  0.9370: 100%|██████████| 500/500 [04:10<00:00,  2.00it/s]
[W 2024-11-29 15:20:34,993] Trial 1 failed with parameters: {'model_name': 'offset5-model', 'num_units': 229, 'learning_rate': 0.00016581411988856763, 'batch_size': 163} because of the following error: RuntimeError('Input type (double) and bias type (float) should be the same').
Traceback (most recent call last):
  File "/home/icaro/Documents/doctorate/dimensionality-reduction/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_91719/3262429375.py", line 15, in experiment
    model.train(learning_rate, batch_size, 500)
  File "/tmp/ipykernel_91719/2048962979.py", line 79, in train
    self.train_decoder(verbose = verbose)
  File "/tmp/ipykernel

RuntimeError: Input type (double) and bias type (float) should be the same